In [ ]:
import time
import pybullet as p
import pybullet_data
import math
import numpy as np

frequence = 240
timeStep = 1/frequence

x1 = 0.27
y1 = 0.16
y2 = 0.32
z1 = -0.1
base_position = [x1,y1,z1]

# Connect to PyBullet and set up the environment
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)
p.setRealTimeSimulation(0)  # Use step simulation for better control
p.setTimeStep(timeStep)  # Set a higher time step for smoother simulation
p.setPhysicsEngineParameter(numSolverIterations=50) 

camera_target_position = [0, 0, 0] # Position to look at
camera_distance = 1 # Distance from the target
camera_yaw = 165 # Yaw angle
camera_pitch = -40 # Pitch angle

# Calculate the camera position
camera_position = [
camera_target_position[0] + camera_distance * math.sin(math.radians(camera_yaw)) * math.cos(math.radians(camera_pitch)),
camera_target_position[1] + camera_distance * math.sin(math.radians(camera_pitch)),
camera_target_position[2] + camera_distance * math.cos(math.radians(camera_yaw)) * math.cos(math.radians(camera_pitch))
]
# Set the camera
p.resetDebugVisualizerCamera(cameraDistance=camera_distance,
cameraYaw=camera_yaw,
cameraPitch=camera_pitch,
cameraTargetPosition=camera_target_position)


# Load the plane and the mill URDF model
plane_id = p.loadURDF("plane.urdf")
robot_id = p.loadURDF("hexapod_2.urdf", useFixedBase=0, basePosition=[0, 0, 0.2], baseOrientation=[0, 0, 0, 1],)
sphere_id = []
for i in range(6):
    sphere_id.append(p.loadURDF("sphere.urdf", useFixedBase=1, basePosition=[0,0,0.1], globalScaling=0.5))

# Joints info
jointIndex = {}
revoluteJointIndex = {}
endEffectorIndex = {}
numJoints = p.getNumJoints(robot_id)
for i in range(numJoints):
    jointInfo = p.getJointInfo(robot_id, i)
    key = jointInfo[1].decode('utf-8')
    jointIndex[key] = i
    if key[0] == 'u' or key[0] == 'v' or key[0] == 'w':
        revoluteJointIndex[key] = i
    elif key[0] == 'x':
        endEffectorIndex[key] = i
N = len(revoluteJointIndex)

# Change dynamics
#p.changeDynamics(plane_id, -1, lateralFriction=1.0)

for idx in revoluteJointIndex.values():
    p.changeDynamics(
        bodyUniqueId=robot_id,
        linkIndex=idx,
        #mass=10,                     # Adjust the mass
        # lateralFriction=0.5,         # Lateral friction coefficient
        # spinningFriction=0.1,        # Torsional friction around the contact normal
        # rollingFriction=0.1,         # Rolling friction orthogonal to contact normal
        # restitution=0.05,            # "Bounciness" coefficient (keep close to 0)
        # linearDamping=0.04,          # Linear damping (resistance to linear movement)
        # angularDamping=0.04,         # Angular damping (resistance to rotation)
        #contactStiffness=1000.0,     # Stiffness of contact constraints
        # contactDamping=0.1,          # Damping of contact constraints
        jointDamping=0.1             # Damping specifically for joints
    )

for idx in endEffectorIndex.values():
    p.changeDynamics(
        bodyUniqueId=robot_id,
        linkIndex=idx,
        #mass=10,                     # Adjust the mass
        lateralFriction=1,         # Lateral friction coefficient
        # spinningFriction=0.1,        # Torsional friction around the contact normal
        # rollingFriction=0.1,         # Rolling friction orthogonal to contact normal
        # restitution=0.05,            # "Bounciness" coefficient (keep close to 0)
        # linearDamping=0.04,          # Linear damping (resistance to linear movement)
        # angularDamping=0.04,         # Angular damping (resistance to rotation)
        contactStiffness=10.0,     # Stiffness of contact constraints
        contactDamping=0.5,          # Damping of contact constraints
        # jointDamping=0.1             # Damping specifically for joints
    )


# Variables
end_effector_world_position = [0,0,0,0,0,0]

# Debug parameters
range_max = 0.5
x1 = 0.16
y2 = 0.24
y1 = 0.16
z1 = -0.07
x1_id = p.addUserDebugParameter("X1", 0 , range_max, x1)
y1_id = p.addUserDebugParameter("Y1", 0 , range_max, y1)
y2_id = p.addUserDebugParameter("Y2", 0 , range_max, y2)
z1_id = p.addUserDebugParameter("Z1", -range_max , 0, z1)
 

t = 0
while(1):
    t = t+1
    if t%5 == 0:
        x1 = p.readUserDebugParameter(x1_id)
        y1 = p.readUserDebugParameter(y1_id)
        y2 = p.readUserDebugParameter(y2_id)
        z1 = p.readUserDebugParameter(z1_id)

        robot_state = p.getLinkState(robot_id,0)
        robot_position = list(robot_state[0])
        robot_orientation = list(robot_state[1])

        base_position = [[x1,y1,z1],[0,y2,z1],[-x1,y1,z1],[-x1,-y1,z1],[0,-y2,z1],[x1,-y1,z1]]

        

        for i in range(6):
            sphere_position, sphere_orientation = p.multiplyTransforms(robot_position,robot_orientation, base_position[i], [0,0,0,1])
            end_effector_world_position[i] = sphere_position
            p.resetBasePositionAndOrientation(sphere_id[i], sphere_position, sphere_orientation)
        
        angles = p.calculateInverseKinematics2(robot_id, endEffectorIndex.values(), end_effector_world_position)
        p.setJointMotorControlArray(robot_id, revoluteJointIndex.values(), p.POSITION_CONTROL, angles, forces=[5]*N, positionGains=[0.5]*N)
        

    p.stepSimulation()
    time.sleep(timeStep)  # Sleep to maintain the desired time step
